In [1]:
### Câu 1 (2 điểm): Cho một ảnh bất kỳ (tên ảnh do sinh viên tự đặt, ví dụ: `my_image.jpg`) và thực hiện các yêu cầu sau:

# * Viết chương trình sử dụng Bilateral filter để làm mịn ảnh. (0.5 điểm)
# * Viết chương trình sử dụng Canny Edge Detection để xác định biên của hình ảnh. (0.5 điểm)
# * Đổi màu ảnh bằng cách hoán đổi kênh màu theo thứ tự (ví dụ: BGR → BRG) và lưu thành tên dạng `[ten_anh]_swapped.jpg`. (0.5 điểm)
# * Chuyển ảnh sang không gian màu YCrCb và tách riêng 3 kênh Y, Cr, Cb, lưu thành ảnh grayscale tương ứng (`[ten_anh]_Y.jpg`, `[ten_anh]_Cr.jpg`, `[ten_anh]_Cb.jpg`). (0.5 điểm)


import cv2
import os
import numpy as np

# Đặt tên ảnh gốc
image_name = 'pagoda.jpg'

# Đọc ảnh
img = cv2.imread(image_name)

if img is None:
    print(f"Không tìm thấy ảnh: {image_name}")
    exit()

# Lấy tên gốc (không có đuôi .jpg)
base_name = os.path.splitext(image_name)[0]

# 1. Làm mịn ảnh bằng Bilateral Filter
bilateral = cv2.bilateralFilter(img, d=9, sigmaColor=75, sigmaSpace=75)
cv2.imwrite(f'{base_name}_bilateral.jpg', bilateral)

# 2. Biên ảnh bằng Canny Edge Detection
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray, threshold1=100, threshold2=200)
cv2.imwrite(f'{base_name}_canny.jpg', edges)

# 3. Đổi kênh màu: ví dụ BGR → BRG
swapped = img.copy()
swapped[:, :, [1, 2]] = swapped[:, :, [2, 1]]  # hoán đổi G và R
cv2.imwrite(f'{base_name}_swapped.jpg', swapped)

# 4. Chuyển sang YCrCb và tách kênh
ycrcb = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
Y, Cr, Cb = cv2.split(ycrcb)

cv2.imwrite(f'{base_name}_Y.jpg', Y)
cv2.imwrite(f'{base_name}_Cr.jpg', Cr)
cv2.imwrite(f'{base_name}_Cb.jpg', Cb)

print("Đã xử lý xong và lưu tất cả ảnh kết quả.")


Đã xử lý xong và lưu tất cả ảnh kết quả.


In [10]:
# 2. Chương trình xử lý đồng thời 3 ảnh bất kỳ do sinh viên tự chọn (có thể chọn bằng đường dẫn file hoặc nhập tên ảnh tùy ý). (0.5 điểm)

# 3. Phím tương ứng để kích hoạt các phương pháp xử lý:
# * Z: Zoom
# * T: Rotate
# * H: Horizontal Flip
# * V: Vertical Flip
# * C: Crop
# * P: Padding (0.5 điểm)

# 4. Lưu file kết quả với định dạng: `result_[phương pháp]_[tên ảnh gốc].jpg`
#    Ví dụ: `result_crop_cat.jpg`, `result_rotate_image1.jpg` (0.5 điểm)

import cv2
import numpy as np
import os
import random

# === Hàm xử lý ===

def read_image(path):
    img = cv2.imread(path)
    if img is None:
        raise FileNotFoundError(f"Không tìm thấy hoặc đọc được ảnh: {path}")
    return img

def zoom_image(img, scale=1.5):
    h, w = img.shape[:2]
    return cv2.resize(img, (int(w * scale), int(h * scale)))

def rotate_image(img):
    angle = random.randint(0, 360)
    h, w = img.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)

    cos = abs(M[0, 0])
    sin = abs(M[0, 1])
    new_w = int(h * sin + w * cos)
    new_h = int(h * cos + w * sin)

    M[0, 2] += (new_w / 2) - center[0]
    M[1, 2] += (new_h / 2) - center[1]

    return cv2.warpAffine(img, M, (new_w, new_h))

def flip_horizontal(img):
    return cv2.flip(img, 1)

def flip_vertical(img):
    return cv2.flip(img, 0)

def crop_center(img):
    h, w = img.shape[:2]
    crop_h = random.randint(int(h * 0.5), int(h * 0.8))
    crop_w = random.randint(int(w * 0.5), int(w * 0.8))
    start_y = (h - crop_h) // 2
    start_x = (w - crop_w) // 2
    return img[start_y:start_y + crop_h, start_x:start_x + crop_w]

def add_padding(img):
    top, bottom = random.randint(10, 40), random.randint(10, 40)
    left, right = random.randint(10, 40), random.randint(10, 40)
    color = [random.randint(0, 255) for _ in range(3)]
    return cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)

# === Bản đồ lựa chọn phím và hàm xử lý ===

operations = {
    'Z': ('zoom', zoom_image),
    'T': ('rotate', rotate_image),
    'H': ('flipH', flip_horizontal),
    'V': ('flipV', flip_vertical),
    'C': ('crop', crop_center),
    'P': ('pad', add_padding)
}

# === Giao diện chính ===

print("📂 Nhập đường dẫn hoặc tên của 3 ảnh muốn xử lý (ảnh phải cùng thư mục hoặc ghi đầy đủ đường dẫn):")
image_paths = []
for i in range(3):
    path = input(f"🔹 Ảnh {i+1}: ").strip()
    if not os.path.exists(path):
        print(f"❌ Không tìm thấy ảnh: {path}")
        exit()
    image_paths.append(path)

# Hiển thị menu thao tác
print("\n🧰 MENU BIẾN ĐỔI ẢNH")
print("Z - Zoom")
print("T - Rotate")
print("H - Horizontal Flip")
print("V - Vertical Flip")
print("C - Crop")
print("P - Padding")
choice = input("🔍 Nhập phím chọn phương pháp (Z/T/H/V/C/P): ").strip().upper()

if choice not in operations:
    print("❗ Lựa chọn không hợp lệ!")
    exit()

# Lấy tên phương pháp và hàm tương ứng
method_name, operation_func = operations[choice]

# Tạo thư mục lưu kết quả
output_folder = 'output_cau2'
os.makedirs(output_folder, exist_ok=True)

# Xử lý từng ảnh
for path in image_paths:
    try:
        img = read_image(path)
        result = operation_func(img)
        # Lấy tên ảnh gốc không có phần mở rộng
        base_name = os.path.splitext(os.path.basename(path))[0]
        out_name = f'result_{method_name}_{base_name}.jpg'
        out_path = os.path.join(output_folder, out_name)
        cv2.imwrite(out_path, result)
        print(f"✅ Đã lưu: {out_name}")
    except Exception as e:
        print(f"❌ Lỗi với ảnh {path}: {e}")



📂 Nhập đường dẫn hoặc tên của 3 ảnh muốn xử lý (ảnh phải cùng thư mục hoặc ghi đầy đủ đường dẫn):
🔹 Ảnh 1: pagoda.jpg
🔹 Ảnh 2: quang_ninh.jpg
🔹 Ảnh 3: colorful-ripe-tropical-fruits.jpg

🧰 MENU BIẾN ĐỔI ẢNH
Z - Zoom
T - Rotate
H - Horizontal Flip
V - Vertical Flip
C - Crop
P - Padding
🔍 Nhập phím chọn phương pháp (Z/T/H/V/C/P): P
✅ Đã lưu: result_pad_pagoda.jpg
✅ Đã lưu: result_pad_quang_ninh.jpg
✅ Đã lưu: result_pad_colorful-ripe-tropical-fruits.jpg


In [6]:
### Câu 3 (4 điểm) Viết một chương trình Python để xử lý 3 ảnh bất kỳ do sinh viên tự chọn.

# * Thêm viền đen 20 pixel cho ảnh đầu tiên. (0.5 điểm)
# * Xoay ảnh thứ hai 45 độ và phóng to 1.5 lần. (0.5 điểm)
# * Tăng kích thước ảnh thứ ba lên 4 lần, sau đó áp dụng Bilateral Filter với tham số tùy chọn. (1.5 điểm)
# * Thay đổi độ sáng và độ tương phản ảnh thứ ba theo công thức:

# $$
# I_{out}(x, y) = \alpha \cdot I_{in}(x, y) + \beta
# $$

# Trong đó:

# $$
# \alpha \in [0.6, 2.0], \quad \beta \in [-60, 60]
# $$

# Giá trị đầu ra cần được giới hạn trong khoảng [0, 255] bằng công thức:

# $$
# I_{out}(x, y) = \text{clip}(I_{out}(x, y), 0, 255)
# $$


import cv2
import numpy as np
import os

# Tên 3 ảnh đầu vào
img1_path = 'colorful-ripe-tropical-fruits.jpg'
img2_path = 'pagoda.jpg'
img3_path = 'quang_ninh.jpg'

# Tạo thư mục lưu kết quả
output_folder = 'output_cau3'
os.makedirs(output_folder, exist_ok=True)

# Hàm đọc ảnh an toàn
def read_image(path):
    img = cv2.imread(path)
    if img is None:
        raise FileNotFoundError(f"Không thể đọc ảnh: {path}")
    return img

# Xử lý ảnh 1: Thêm viền đen 20 pixel
def process_image1(img):
    bordered = cv2.copyMakeBorder(img, 20, 20, 20, 20, cv2.BORDER_CONSTANT, value=(0, 0, 0))
    return bordered

# Xử lý ảnh 2: Xoay 45 độ và phóng to 1.5 lần
def process_image2(img):
    scale = 1.5
    angle = 45
    h, w = img.shape[:2]
    center = (w // 2, h // 2)

    M = cv2.getRotationMatrix2D(center, angle, scale)

    # Tính kích thước mới để tránh mất góc
    cos = abs(M[0, 0])
    sin = abs(M[0, 1])
    new_w = int((h * sin + w * cos) * scale)
    new_h = int((h * cos + w * sin) * scale)

    # Điều chỉnh tịnh tiến để không bị cắt ảnh
    M[0, 2] += (new_w / 2) - center[0]
    M[1, 2] += (new_h / 2) - center[1]

    rotated = cv2.warpAffine(img, M, (new_w, new_h))
    return rotated

# Xử lý ảnh 3:
def process_image3(img, alpha=1.2, beta=30):
    # Phóng to 4 lần
    h, w = img.shape[:2]
    zoomed = cv2.resize(img, (w * 4, h * 4), interpolation=cv2.INTER_LINEAR)

    # Áp dụng Bilateral Filter (tham số tùy chọn)
    bilateral = cv2.bilateralFilter(zoomed, d=9, sigmaColor=75, sigmaSpace=75)

    # Thay đổi độ sáng và tương phản
    output = cv2.convertScaleAbs(bilateral, alpha=alpha, beta=beta)

    return output

# Đọc ảnh và xử lý
try:
    img1 = read_image(img1_path)
    result1 = process_image1(img1)
    cv2.imwrite(os.path.join(output_folder, 'colorful-ripe-tropical-fruits.jpg'), result1)
    print("Đã xử lý ảnh 1: thêm viền đen.")

    img2 = read_image(img2_path)
    result2 = process_image2(img2)
    cv2.imwrite(os.path.join(output_folder, 'pagoda.jpg'), result2)
    print("Đã xử lý ảnh 2: xoay 45 độ và phóng to.")

    img3 = read_image(img3_path)
    result3 = process_image3(img3, alpha=1.5, beta=40)
    cv2.imwrite(os.path.join(output_folder, 'quang_ninh.jpg'), result3)
    print("Đã xử lý ảnh 3: phóng to, lọc Bilateral, chỉnh sáng tương phản.")

except FileNotFoundError as e:
    print(str(e))



Đã xử lý ảnh 1: thêm viền đen.
Đã xử lý ảnh 2: xoay 45 độ và phóng to.
Đã xử lý ảnh 3: phóng to, lọc Bilateral, chỉnh sáng tương phản.
